## sparkmlhw04
采用ML Pipelines构建一个文档分类器，需要进行文档清洗、提取文档关键字、得到文档向量、提取分类主题的关键字，最后建立一个文档主题目分类器，进行模型训练、测试、分类指标：

数据：
data_app

数据格式：
doc_id1| doc_id2| soure|name|doc_name|country| typename|text

其中文档ID字段为：doc_id1
其中文档名称字段为：doc_name
其中文档类别字段为：typename
其中文档内容为：text

采用ML Pipelines构建一个文档分类器，需要进行文档清洗、提取文档关键字、得到文档向量、提取分类主题的关键字，最后建立一个文档主题目分类器，进行模型训练、测试、分类指标：
1）	文档清洗
2）	文档关键字提取
3）	主题的关键字提取
4）	文档向量化，采用3种向量方法，进行测试
5）	建立主题分类器
6）	训练、测试、指标


In [28]:
import pyspark
from pyspark.sql import SparkSession
#生成SparkSession实例
spark = SparkSession.builder \
     .master("local[*]") \
     .appName("sparkmlhw04") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/ian/installed/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ian/installed/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/home/ian/installed/anaconda3/lib/python3.6/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling o30.sessionState

In [5]:
df1 = spark.read.csv("file:///home/ian/code/data/sparkml/data_app", sep='|', header=False)

In [11]:
df2 = df1.withColumnRenamed('_c0','doc_id1')\
    .withColumnRenamed('_c1','doc_id2')\
    .withColumnRenamed('_c2','soure')\
    .withColumnRenamed('_c3','name')\
    .withColumnRenamed('_c4','doc_name')\
    .withColumnRenamed('_c5','country')\
    .withColumnRenamed('_c6','typename')\
    .withColumnRenamed('_c7','text')

In [12]:
df2.show(5)

+-------+-------+-----+--------------------+--------------------+-------+----------------+--------------------+
|doc_id1|doc_id2|soure|                name|            doc_name|country|        typename|                text|
+-------+-------+-----+--------------------+--------------------+-------+----------------+--------------------+
|1494604| 549745|    7|Curriculum Vitae-...|curriculum.vitae....|     en|        Business|My professional r...|
|1494602| 604435|    7|Five Questions (S...|com.kevinhecker.t...|     en|          Puzzle|Five Questions is...|
|1494599| 495855|    7|      RhymasaurusRex|com.yeti.rhymasau...|     en|   Entertainment|Are you a songwri...|
|1494596| 714480|    7|            helphelp|com.triplekingkon...|     en|       Lifestyle|guardian angel %2...|
|1494593| 568853|    7|Calorie Intake Ca...|com.piusvelte.cal...|     en|Health & Fitness|Description Estim...|
+-------+-------+-----+--------------------+--------------------+-------+----------------+--------------

In [14]:
#fenci
from pyspark.ml.feature import RegexTokenizer
rtokenizer = RegexTokenizer(inputCol='text', outputCol='words')
rtokenizer.setPattern('[, ]+')
df3 = rtokenizer.transform(df2)
df3.show(5)

+-------+-------+-----+--------------------+--------------------+-------+----------------+--------------------+--------------------+
|doc_id1|doc_id2|soure|                name|            doc_name|country|        typename|                text|               words|
+-------+-------+-----+--------------------+--------------------+-------+----------------+--------------------+--------------------+
|1494604| 549745|    7|Curriculum Vitae-...|curriculum.vitae....|     en|        Business|My professional r...|[my, professional...|
|1494602| 604435|    7|Five Questions (S...|com.kevinhecker.t...|     en|          Puzzle|Five Questions is...|[five, questions,...|
|1494599| 495855|    7|      RhymasaurusRex|com.yeti.rhymasau...|     en|   Entertainment|Are you a songwri...|[are, you, a, son...|
|1494596| 714480|    7|            helphelp|com.triplekingkon...|     en|       Lifestyle|guardian angel %2...|[guardian, angel,...|
|1494593| 568853|    7|Calorie Intake Ca...|com.piusvelte.cal...|    

In [15]:
#pyspark.ml.feature.StopWordsRemover
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="rwords", stopWords=StopWordsRemover.loadDefaultStopWords('english'))
df4 = remover.transform(df3)
df4.show(2)

+-------+-------+-----+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|doc_id1|doc_id2|soure|                name|            doc_name|country|typename|                text|               words|              rwords|
+-------+-------+-----+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
|1494604| 549745|    7|Curriculum Vitae-...|curriculum.vitae....|     en|Business|My professional r...|[my, professional...|[professional, re...|
|1494602| 604435|    7|Five Questions (S...|com.kevinhecker.t...|     en|  Puzzle|Five Questions is...|[five, questions,...|[five, questions,...|
+-------+-------+-----+--------------------+--------------------+-------+--------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [21]:
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(numFeatures=1000, inputCol='rwords', outputCol='features')
df5 = hashingTF.transform(df4)
df5.select('features').show(1, False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(1000,[69,70,102,124,144,155,159,160,169,239,242,256,265,322,338,356,364,399,411,522,537,556,586,621,6

In [17]:
#pyspark.ml.feature.Word2Vec
from pyspark.ml.feature import Word2Vec

wv = Word2Vec(vectorSize=128, inputCol='rwords', outputCol='wvfeatures')
wvModel = wv.fit(df4)

In [23]:
wvModel.getVectors().show(1, False)

NameError: name 'wvModel' is not defined

In [35]:
df6 = wvModel.transform(df3)
df6.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          wvfeatures|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|[0.02366950090120...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [27]:
#pyspark.ml.feature.CountVectorizer
from pyspark.ml.feature import CountVectorizer


cv = CountVectorizer(vocabSize=10, inputCol='rwords', outputCol='cvfeatures')
cvModel = cv.fit(df4)



Py4JJavaError: An error occurred while calling o605.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 13 in stage 22.0 failed 1 times, most recent failure: Lost task 13.0 in stage 22.0 (TID 455, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$10$$anon$1.hasNext(WholeStageCodegenExec.scala:614)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at org.apache

In [ ]:
df7 = cvModel.transform(df3)
df7.select('cvfeatures').show(1)

In [9]:
from pyspark.ml.clustering import LDA

In [40]:
lda = LDA(k=5)
ldaModel = lda.fit(df4)

In [44]:
ldatransformed = ldaModel.transform(df4)
ldatransformed.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|            features|   topicDistribution|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|(1000,[10,51,68,7...|[0.00407877175527...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [46]:
df5.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          wvfeatures|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|[0.02366950090120...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [47]:
lda1 = LDA(k=5, featuresCol='wvfeatures')
ldaModel1 = lda1.fit(df5)
ldatransformed1 = ldaModel1.transform(df5)
ldatransformed1.show(1)

Py4JJavaError: An error occurred while calling o721.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 76.0 failed 1 times, most recent failure: Lost task 2.0 in stage 76.0 (TID 170, localhost, executor driver): java.lang.StackOverflowError
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:445)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma(Gamma.java:460)
	at org.apache.commons.math3.special.Gamma.digamma

In [11]:
lda2 = LDA(k=5, featuresCol='cvfeatures')
ldaModel2 = lda2.fit(df6)
ldatransformed2 = ldaModel2.transform(df6)
ldatransformed2.show(1)

+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|myapp_id|typenameid|typename|          myapp_word|      myapp_word_all|               words|              rwords|          cvfeatures|   topicDistribution|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1376533|         2|  action|game, android, world|game, android, wo...|[game, android, w...|[game, android, w...|(510,[0,1,2,3,4,5...|[0.00267203027568...|
+--------+----------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row

